In [21]:
pd.read_csv('data_for_scoring_wallstreetbets_2016-01-01_2016-06-01_True_Casey.csv').shape

(205, 6)

In [18]:
%%writefile make_scoring_data.py

start_date = '2016-01-01'
end_date = '2016-06-01'
dollar_sign = True
subreddit = 'wallstreetbets'
total_N = 600

fname = 'data_for_scoring_{0}_{1}_{2}_{3}_{4}.csv'

import sys
sys.path.insert(0, '../..')
from RETF.reddit_scraper.scrape import SubScraper
from RETF.portfolio.get_stats import Post, unix_to_datetime
import pandas as pd
from RETF.portfolio.match_tickers import SymbolFinder
from RETF.portfolio.index import make_ticker_list
import numpy as np


def format_labeling_data(submission):
    p = Post(submission)
    # create text the same way it will be created in prod
    p.create_text() 
    # return filtered tokens
    return unix_to_datetime(submission.created_utc), p.text, sf.get_primary_ticker(p.text), 'unk'

if __name__ == '__main__':

    sf = SymbolFinder(make_ticker_list(['../../data/nyse.csv', '../../data/nasdaq.csv']), dollar_sign=dollar_sign)
    
    scraper = SubScraper("../../RETF/credentials.txt")
    
    submissions=scraper.get_submissions_between(subreddit, start_date, end_date)
    
    df = pd.DataFrame([format_labeling_data(x) for x in submissions], columns=['date', 'text', 'ticker', 'score'])
    
    df = df.loc[df.ticker.notnull()]
    
    df = df.sample(total_N)
    
    df['scorer'] = np.random.choice(['Andrew','Joe','Casey'], len(df))
    
    for person in ['Andrew', 'Joe', 'Casey']:
        df[df.scorer == person].to_csv(fname.format(subreddit, start_date, end_date, dollar_sign, person))

Overwriting make_scoring_data.py


## Script for labelling

In [17]:
%%writefile score_file.py

# If you quit before completing the exercise, you point the `infile` 
# argument to the `outfile` so you may pick up where you left off.

# the file for scoring
infile = "data_for_scoring_wallstreetbets_2016-01-01_2016-06-01_True_Andrew.csv"
# the output file (can be same as input file)
outfile = "output_andrew.csv"

import pandas as pd

def print_message(row):
    print("\n-------------------")
    print(row.text)
    print("")
    print("Symbol: {0}".format(row.ticker))
    print("")
    print("1: Positive (bullish)")
    print("2: Negative (bearish)")
    print("3: Not Sure/Wrong Ticker")
    print("4: Exit")
    

if __name__ == '__main__':
    
    df = pd.read_csv(infile, index_col=0)
    
    for ix, row in df.iterrows():
        if row.score == 'unk':
            while True:
                print_message(row)
                inp = input("Input: ")
                if not inp in ['1', '2', '3', '4']:
                    print('thats not valid, try again')
                else:
                    break
            if inp == '4':
                break
            df.loc[ix, 'score'] = ['bull', 'bear', 'usr_unk'][int(inp)-1]

    df.to_csv(outfile)

Overwriting score_file.py


## After they go off for labelling, bring them back in and convert for modelling

In [ ]:

from RETF.portfolio.match_tickers import SymbolFinder
sf = SymbolFinder(make_ticker_list(['../../data/nyse.csv', '../../data/nasdaq.csv']), dollar_sign=dollar_sign)


def make_modeling_data(text):
    # remove possible tickers
    tokens = sf.tokenize(p.text) 
    possible_tickers = sf.filter_symbols(tokens)
    [x.lower().strip() for x in tokens if x not in possible_tickers]